## Dependencies

In [1]:
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
model_path = '/kaggle/input/112-googleq-a-train-bert-base-uncased-seq-slices/model.h5'
BERT_PATH = '/kaggle/input/tf-hub-bert-base/bert_base_uncased'
VOCAB_PATH = BERT_PATH + '/assets/vocab.txt'

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())
    # Map misspellings
    test[feature] = test[feature].apply(lambda x: map_misspellings(x))
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

bert_layer = KerasLayer(BERT_PATH, trainable=True)
pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

t = GlobalAveragePooling1D()(sequence_output[:,:32])
t = Dense(30, kernel_initializer='glorot_uniform', activation="relu")(t)

q = GlobalAveragePooling1D()(sequence_output[:,32:272])
q = Dense(239 , kernel_initializer='glorot_uniform', activation="relu")(q)

a = GlobalAveragePooling1D()(sequence_output[:,272:])
a = Dense(239, kernel_initializer='glorot_uniform', activation="relu")(a)

x = Concatenate()([t, q, a])
x = Dropout(0.5)(x)
output = Dense(N_CLASS, kernel_initializer='glorot_uniform', activation="sigmoid", name="output")(x)

model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)

model.summary()

model.load_weights(model_path)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_masks[0][0]            

# Make predictions

In [8]:
Y_test = model.predict(X_test)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.921066,0.610168,0.144465,0.448082,0.672214,0.620355,0.612001,0.595575,0.681730,...,0.886041,0.893564,0.562945,0.958361,0.943624,0.787837,0.070038,0.061282,0.872571,0.935384
1,46,0.869215,0.507531,0.010229,0.698362,0.808485,0.902444,0.604737,0.418581,0.201691,...,0.777552,0.956792,0.659844,0.965321,0.971986,0.881013,0.908154,0.136443,0.171373,0.910410
2,70,0.910272,0.656429,0.028114,0.509277,0.915863,0.877950,0.572807,0.487129,0.419414,...,0.891087,0.914009,0.533682,0.961720,0.960656,0.841007,0.107510,0.072364,0.825303,0.932896
3,132,0.857790,0.400944,0.014626,0.680031,0.843541,0.892618,0.573563,0.389045,0.072452,...,0.664500,0.937674,0.663459,0.969819,0.968089,0.910400,0.798057,0.134580,0.633274,0.914932
4,200,0.890479,0.428216,0.044727,0.745125,0.766546,0.802076,0.670512,0.556969,0.142475,...,0.717912,0.905510,0.604958,0.953822,0.952739,0.803458,0.312878,0.168383,0.730955,0.922825


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.887195,0.570038,0.038168,0.593344,0.814008,0.802008,0.609381,0.483115,0.290513,...,0.801824,0.920365,0.641884,0.959364,0.957868,0.863350,0.536090,0.136922,0.529359,0.919569
std,2812.670060,0.037265,0.129428,0.045787,0.126020,0.098639,0.133110,0.048751,0.083025,0.216029,...,0.079106,0.028634,0.049399,0.013885,0.015237,0.043681,0.332179,0.061333,0.288227,0.016882
min,39.000000,0.778518,0.343606,0.006666,0.170822,0.371704,0.250369,0.481933,0.328770,0.019025,...,0.581190,0.796710,0.507726,0.897104,0.890632,0.727685,0.018531,0.020245,0.026099,0.874708
25%,2572.000000,0.863683,0.461994,0.015622,0.513732,0.761051,0.761324,0.573417,0.425685,0.105877,...,0.737734,0.904427,0.608845,0.951019,0.949771,0.833818,0.179708,0.088347,0.268265,0.906888
50%,5093.000000,0.889840,0.554674,0.021440,0.600633,0.817715,0.847612,0.604183,0.466340,0.216536,...,0.806389,0.925441,0.641634,0.961409,0.959934,0.869298,0.601120,0.138011,0.535550,0.918400
75%,7482.000000,0.914302,0.677408,0.036069,0.671533,0.888289,0.887411,0.643631,0.529660,0.441356,...,0.870766,0.940094,0.675726,0.969680,0.968645,0.894910,0.856297,0.176337,0.790279,0.930604
max,9640.000000,0.966313,0.870361,0.380407,0.892906,0.980710,0.967770,0.749178,0.798187,0.833031,...,0.959318,0.973849,0.789110,0.985801,0.987722,0.945174,0.974709,0.311538,0.980750,0.965638
